<h3>
Finetuning clusters themewise
</h3>

Overview: We already have divided the ~650 themes in squad into 21 clusters(best according to elbow method). Here for every theme given in a question, we are finetuning the previous saved finetuned models using data from each theme - capped to 300 data for each theme by synthesizing new data from the existing data.


<p>For more details please head over to the comments </p>


Requirements:
1. cluster.json in gdrive: https://drive.google.com/file/d/1FXM9SxGJ7o_DM_l7__5T7GxFywngxhi0/view?usp=share_link
2. final_train in gdrive: https://drive.google.com/file/d/1TX35oLZEx_VgSa6I-Oe1-EhnuDQ5JB3G/view?usp=share_link


In [ ]:
"""
Book Keeping: 
1. Cluster.json file should be present in drive
2. org_df = pd.read_csv('/content/drive/MyDrive/train_data.csv') 
3. # callbacks=callbacks, --> change the callbacks accordingy
4. rows_req = 300 Total no of rows reqd --> best if it is 250, try others in higher units
5. # callbacks = [cp_callback] change accordingly shall you wish to save the weights manually
"""

"\nBook Keeping: \n1. Cluster.json file should be present in drive\n2. org_df = pd.read_csv('/content/drive/MyDrive/train_data.csv') \n3. # callbacks=callbacks, --> change the callbacks accordingy\n4. rows_req = 300 Total no of rows reqd --> best if it is 250, try others in higher units\n5. # callbacks = [cp_callback] change accordingly shall you wish to save the weights manually\n"

In [ ]:
"""
Requirement of HuggingFace hub to store the trained models
"""
! pip install transformers datasets huggingface_hub

# logging in
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# ------------------------------------------All installations here------------------------------------------


from google.colab import drive
drive.mount('/content/drive')

# git installation
!apt install git-lfs
!git config --global user.email "nandysoham16@gmail.com"
!git config --global user.name "nandysoham"


# all imports
import tensorflow as tf
import transformers
from datasets import load_dataset, load_metric
import pandas as pd
import numpy as np
import random
from transformers import AutoTokenizer
import transformers
from transformers import TFAutoModelForQuestionAnswering
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
from tensorflow.keras.callbacks import TensorBoard
import collections
from transformers import create_optimizer

# choose a random theme
!pip3 install unidecode
from unidecode import unidecode
from datasets import load_dataset, load_metric, Dataset

import json
with open('/content/drive/MyDrive/cluster.json') as json_file:                  
    clusters_dict = json.load(json_file)


clustered = clusters_dict["clusters"]
centroids = clusters_dict["centroids"]

!pip install sentence_transformers
from sentence_transformers import SentenceTransformer



Mounted at /content/drive
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 27 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 4.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.0 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=c552e9f2bf55ae4a6deb6d408d4aa274826

In [ ]:
# --------------------All other models that are indirectly in use ----------------------#

clustering_model = SentenceTransformer('all-MiniLM-L6-v2')

# Augemeters: ref: NLPAUG
!pip install nlpaug
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

#  ref: https://github.com/makcedward/nlpaug/blob/master/example/tfidf-train_model.ipynb

from nlpaug.util import Action

import os
os.environ["MODEL_DIR"] = '../model'
model_dir = '../model'


import sklearn.datasets
import re

import nlpaug.augmenter.word as naw
import nlpaug.model.word_stats as nmw

def _tokenizer(text, token_pattern=r"(?u)\b\w\w+\b"):
    token_pattern = re.compile(token_pattern)
    return token_pattern.findall(text)

# Load sample data
train_data = sklearn.datasets.fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
train_x = train_data.data

# Tokenize input
train_x_tokens = [_tokenizer(x) for x in train_x]

# Train TF-IDF model
tfidf_model = nmw.TfIdf()
tfidf_model.train(train_x_tokens)
tfidf_model.save('.')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 KB 7.4 MB/s eta 0:00:00


In [ ]:
augmenter = ["delete", "wordnet","word2vec", "TFID", "antonym"]


def augment_text_TFID(text):
  """
  TfID augmenter
  """
 # Load TF-IDF augmenter
  aug = naw.TfIdfAug(model_path='.', tokenizer=_tokenizer, action="substitute")  # insert, substitute and other options exist
  augmented_text = aug.augment(text)
  # print("Original:")
  # print(text)
  # print("Augmented Text:")
  # print(augmented_text)
  return augmented_text


def augment_text_wordnet(text):
  """
  synonym generator form wordnet
  """
  aug = naw.SynonymAug(aug_src='wordnet')
  augmented_text = aug.augment(text)
  return augmented_text


def augment_text_word2vec(text):
  """
  Substitutes from word2vec model
  """
  aug = naw.WordEmbsAug(
    model_type='word2vec', model_path='/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300',
    action="substitute")
  augmented_text = aug.augment(text)
  return augmented_text


def augment_text_antonym(text):
  """
  substitus antonyms
  """
  aug = naw.AntonymAug()
  augmented_text = aug.augment(text)
  return augmented_text
  
def augment_text_delete(text):
  """
  Delete random text
  """
  aug = naw.RandomWordAug()
  augmented_text = aug.augment(text)
  return augmented_text


def augment_text(text):

  rand_no = random.randint(2,5)
  augmented_list = random.choices(augmenter, k = rand_no)
  print(augmented_list)
  for layer in augmenter:
    if(layer == "wordnet"):
      text = augment_text_wordnet(text)

    if(layer == "TFID"):
      text = augment_text_TFID(text)

    if(layer == "word2vec"):
      test = augment_text_word2vec(text)

    if(layer == "antonym"):
      text = augment_text_antonym(text)

    if(layer == "delete"):
      text = augment_text_delete(text)
    return text
  # print("error in program")

In [ ]:
def convert_to_squad(df):
  """
  INPUTS : DF with the given inputs
  ------------------------------------------------------------------------------
  Unknown_col     Theme   Question  Paragraph   Anser_text     Answer_start
  ------------------------------------------------------------------------------


  OUTPUT: DF WITH SQUADLIKE COLUMNS
  ------------------------------------------------------------------------------
  id      title     context     question    answers{"text", "answer_start"}
  ------------------------------------------------------------------------------



  """
  # first pick up only those rows which have an answer
  df_onlyposs = df[df.Answer_possible == True ]
  print(df_onlyposs)
  # print(type(df_onlyposs["Theme"]))
  theme = np.array(df_onlyposs["Theme"])
  question = np.array(df_onlyposs["Question"])
  paragraph = np.array(df_onlyposs["Paragraph"])
  answer_text = np.array(df_onlyposs["Answer_text"])
  answer_start = np.array(df_onlyposs["Answer_start"])

  print(type(answer_text))
# 
  answer = np.array([{"text" : [answer_text[i]] , "answer_start" : [answer_start[i]]} for i in range(len(answer_text))])
  # print(answer)
  columns = ["id", "title", "context", "question", "answers"]
  squad_dict = [[df_onlyposs.iloc[i][0],theme[i],paragraph[i],question[i],answer[i]] for i in range(len(theme))]
  squad_df = pd.DataFrame(squad_dict, columns = columns)
  # print(squad_df)
  return squad_df




def load_dataset():
  global datasets
  org_df = pd.read_csv('/content/drive/MyDrive/final_train.csv')                             # CHANGE THE URL TO THE ORIGINAL DATA
  squad_df = convert_to_squad(org_df)
  # return squad_df
  datasets = Dataset.from_pandas(squad_df)
  # print(datasets)
  return datasets

load_dataset() 

     Unnamed: 0                                           Question  \
0             0                   Which company produces the iPod?   
1             1               When was the original iPod released?   
2             2  How many different types of iPod are currently...   
3             3                   What kind of device is the iPod?   
4             4        The iPod Touch uses what kind of interface?   
..          ...                                                ...   
939         939      What type of songs was Leona Lewis known for?   
940         940  Along with mainstream AC, what format has also...   
941         941  What AC format is especially known for playing...   
942         942  When is the later of the two times that Christ...   
943         943  When is the earliest an AC station will switch...   

                        Theme  paragraph_id                Answer_text  \
0                        IPod             1                      Apple   
1          

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 944
})

In [ ]:
# parameters for the data and the model
squad_v2 = False
batch_size = 16
model_checkpoint = ""


from traitlets.config import T
import numpy as np

# 140 S
def Generate_Synthetic_Data(theme_df,theme, count):
  """
  INPUTS: DF theme_df
          theme --> particularly for preparing moe samples
          count --> No. of extra points required

  OUTPUTS: synthetic_list_augmented --> data after reodering and augmentation

            , synthetic_list --> data just after reordering

  
  Working:


  REORDERING (Albumentation): 

  1. Given a passage ,split it by '.'s 
  2. Let the answer be passage[i...j] that is from index i to index j
  3. Take index of stop (.) just before i and just after j, let then be i' and j'
  3. Keep this part intact
  4. now reorder all sentences and this shall be the new context

  Augmentation:
  1. after reordering, keeping the new part arising from part from [i...j] intact, chose the part above it [0..f(i')-1] and after it [f{j')+1]. - f{x} reoreded value of x
  2. Now try to augment the data using standard nlp augmentation techniques
  3. from 5 options, a random subset is chosen, as doing all 5 will be computationally expensive and the data will lose randomness
  4. Now the final augmented data is ready

  """
  synthetic_list = []
  synthetic_list_augmented =[]
  extra_item = 0
  force_stop = False  # if in case no new dats can be generated

  while(len(synthetic_list) < count and not force_stop):
    # print("Here")
    # print(len(synthetic_list))
    stop = True
    for ind in theme_df.index:
      # print(ind)
      if(len(synthetic_list) > count):
        break

      context = theme_df["context"][ind]
      isposs = True
      answer_start = -1
      answer_end = -1
      text = ""
      question = theme_df["question"][ind]
      if(len(theme_df["answers"][ind]["answer_start"]) > -1):
        text = theme_df["answers"][ind]["text"][0]
        text_size = len(text)
        answer_start = theme_df["answers"][ind]["answer_start"][0]
        answer_end = answer_start + text_size - 1
      else:
        isposs = False

      # print(context)
      # print(text)
      # print(answer_start)
      # print(answer_end)

      # if(isposs):
      # problem here 12.35 --> will also get split
      sentence_list = context.split(".")

      # breaking the whole sentence
      # print(sentence_list)

      # each entry --> [start_index_of_the_sentence , end_index_of_the_sentence(includes .)]
      sentence_index_list = []

      # mask[0] = 0 --> answer not present, else present
      sentence_mask = []
      prev = 0;


      for sentence in sentence_list:
        sentence_index_list.append([prev, prev+len(sentence)])
        prev = prev+len(sentence)+1

      # ============= open up to debug ==============
      # print(sentence_index_list)

      # for lst in sentence_index_list:
      #   print(context[lst[0]:lst[1] + 1], sep=",")
      # print()

      for pr in sentence_index_list:
        if(pr[0] > answer_end or pr[1] < answer_start):
          sentence_mask.append(0)
        else:
          sentence_mask.append(1)

      # print(sentence_mask)
      n = len(sentence_mask)

      # these are final after combining every sentence where mask = 1
      sentence_mask_final = []
      sentence_index_list_final = []
      i =0
      offset = 0

      # combining the ones together
      while(i < n):
        if(sentence_mask[i] == 0):
          sentence_mask_final.append(0)
          sentence_index_list_final.append(sentence_index_list[i])
        else:
          start_idx = sentence_index_list[i][0]
          offset = answer_start - start_idx
          end_idx = sentence_index_list[i][1]
          while(i < n and sentence_mask[i] == 1):
            end_idx = sentence_index_list[i][1]
            i+=1
          if(i < n):
            i-=1
          sentence_mask_final.append(1)
          sentence_index_list_final.append([start_idx, end_idx])
        i+=1
      # print(sentence_mask_final)
      # print(sentence_index_list_final)

      n = len(sentence_mask_final)
      reordered = np.arange(n)
      np.random.shuffle(reordered)
      # print(reordered)
      corpus = ""

      # using nlpaug to generate augmented data
      ans_index = -1
      before_ans = ""
      after_ans = ""
      actual_ans_sentence = ""

      if(isposs):
        before = 0
        passed_answer = False
        for lst in reordered:
          if(not passed_answer):
            if(sentence_mask_final[lst] == 0):
              before += sentence_index_list_final[lst][1] - sentence_index_list_final[lst][0] + 1
              before_ans += context[sentence_index_list_final[lst][0] : sentence_index_list_final[lst][1] + 1]
            else:
              ans_index = before + offset
              actual_ans_sentence += context[sentence_index_list_final[lst][0] : sentence_index_list_final[lst][1] + 1]
              passed_answer = True

          else:
            after_ans += context[sentence_index_list_final[lst][0] : sentence_index_list_final[lst][1] + 1]
          corpus += context[sentence_index_list_final[lst][0] : sentence_index_list_final[lst][1] + 1]
        pass
      else:
        for lst in reordered:
          corpus += context[sentence_index_list_final[lst][0] : sentence_index_list_final[lst][1] + 1]
          before_ans += context[sentence_index_list_final[lst][0] : sentence_index_list_final[lst][1] + 1]


      # print("before ans --> " ,before_ans)
      # print("after_ans -->" , after_ans)
      # print("actual ans sentence --> ",actual_ans_sentence)

      before_ans = augment_text(before_ans)[0] if (len(augment_text(before_ans)) > 0 )  else ""
      after_ans = augment_text(after_ans)[0] if (len(augment_text(after_ans)) > 0 )  else ""

      augmented_ans_index = -1
      if(len(text)):
        augmented_ans_index = len(before_ans) + offset

      augmented_corpus = before_ans + actual_ans_sentence + after_ans
      entry = {}
      entry["id"] = int(np.random.rand()*1e5)
      entry["title"] = theme
      entry["context"] = augmented_corpus
      entry["question"] = question
      entry["answers"] = {"text" : [text if len(text) else ""], "answer_start": [augmented_ans_index] if augmented_ans_index > -1 else []}
      # print("org ans = ", text)
      # if(augmented_ans_index != -1):
      #   print("ans = ", augmented_corpus[augmented_ans_index : augmented_ans_index + len(text)])
      # print(entry)

      synthetic_list_augmented.append(entry)
      ######## ------------- open this up to create just normal - non augmented data --------------#########
      entry = {}
      entry["id"] = int(np.random.rand()*1e5)
      entry["title"] = theme
      entry["context"] = corpus
      entry["question"] = question
      entry["answers"] = {"text" : [text if len(text) else ""], "answer_start": [ans_index] if ans_index > -1 else []}

      # print("-----------------------------")
      # print(context)
      # print(corpus)
      # print(text)
      # print(ans_index)
      # print(corpus[ans_index : ans_index + len(text)])
      # print("-----------------------------")
      
      prev_len = len(synthetic_list)
      synthetic_list.append(entry)
      up_len = len(synthetic_list)

      if(prev_len != up_len):
        stop = False

    if(stop == True):
      force_stop = True

    # break
  return synthetic_list_augmented , synthetic_list

def prepare_synthetic_dataset(theme_df, theme, length, join=True, mix=True):
  """
  INPUT: theme_df - squad_like df corresponding to a theme
  OUTPUT : synthetic_df, syntthtic_df_augmented --> DFs refined after passing the various lengths and options


  acts a layer on top of the Generate_synthetic_dat function --> assing the various parameters
  """

  # length --> will denote no of EXTRA ROWS WE NEED
  if(length <= 0):
    return theme_df, theme_df

  synthetic_list_augmented, synthetic_list = Generate_Synthetic_Data(theme_df, theme, length )      #1000 - len(theme_df)
  synthetic_df = pd.DataFrame(synthetic_list)

  if(join == True):
    for ind in range(len(theme_df)):
      synthetic_df.loc[len(synthetic_df)]= theme_df.iloc[ind]

  if(mix == True):
    synthetic_df = synthetic_df.sample(frac = 1)

  synthetic_df_augmented = pd.DataFrame(synthetic_list_augmented)
  if(join == True):
    for ind in range(len(theme_df)):
      synthetic_df_augmented.loc[len(synthetic_df_augmented)]= theme_df.iloc[ind]
  
  if(mix == True):
    synthetic_df_augmented = synthetic_df_augmented.sample(frac = 1)
  
  return synthetic_df, synthetic_df_augmented


sqaud_v1_dataset_dict = {}

def split_by_themes(df_squad_v1):
  """
  Utility for decomposing the whole of the dataset by themes
  """
  global squad_v1_dataset_dict
  # extracting the themes
  themes = df_squad_v1.title.unique()
  squad_v1_df_dict = {elem : pd.DataFrame() for elem in themes}
  for key in squad_v1_df_dict.keys():
    squad_v1_df_dict[key] = df_squad_v1[:][df_squad_v1.title == key]

  # print(themes)
  squad_v1_dataset_dict = {}
  for key in squad_v1_df_dict.keys():
    squad_v1_dataset_dict[key] = Dataset.from_pandas(squad_v1_df_dict[key])
  return squad_v1_dataset_dict



def create_dataset( **kwargs):
  """
  INPUT: in kwargs;
  theme --> theme
  augment_to --> shall you want augmented + synthetic data
  shall_mix --> shall the data needs to be reordered by indices
  only_aug --> shall you wnat a mixture of the original + synthetic data or only synthetic data

  
  OUTPUT:
  datasets for train and test
  """
 
  theme = kwargs["theme"]
  rows_req = 300


  # augmenting parameters
  mixing = True
  only_aug = False
  shall_augment = False
  datasets_org = squad_v1_dataset_dict[theme]
  datasets_org = datasets_org.train_test_split(test_size=0.1)

  theme_df = pd.DataFrame(datasets_org["train"])
  unaugmented_train_dataset = datasets_org["train"]
  test_dataset = datasets_org["test"]
  

  # augmenting parameters
  if("augment_to" in kwargs.keys()):
    shall_augment = True
    rows_req = kwargs["augment_to"]

    if("shall_mix" in kwargs.keys()):
      mixing = kwargs["shall_mix"]

    if("only_aug" in kwargs.keys()):
      shall_join = kwargs["only_aug"]



  if(only_aug == False):
    rows_req -= len(theme_df.index)
  
  if(shall_augment == False):
    return unaugmented_train_dataset, test_dataset
    
  else:
    if(rows_req <= 0):
      synthetic_df_augmented = theme_df[:len(theme_df.index) + rows_req]
    else:
      synthetic_df, synthetic_df_augmented = prepare_synthetic_dataset(theme_df, theme,rows_req,not only_aug, mixing)
    print(synthetic_df_augmented)
    train_dataset = Dataset.from_pandas(synthetic_df_augmented)
    return train_dataset, test_dataset

  

In [ ]:
# ----------------------------- ACTUAL MODEL ------------------------------

# hyperparameters

pad_on_right = True
max_length = 384  # The maximum length of a feature (question and context)
doc_stride = 128  # The allowed overlap between two part of the context when splitting is performed.



# running the actual model
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    # print(tokenizer)
    

    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (
                offsets[token_start_index][0] <= start_char
                and offsets[token_end_index][1] >= end_char
            ):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while (
                    token_start_index < len(offsets)
                    and offsets[token_start_index][0] <= start_char
                ):
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples




def prepare_validation_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

from tqdm.auto import tqdm

def postprocess_qa_predictions(
    examples,
    features,
    all_start_logits,
    all_end_logits,
    n_best_size=20,
    max_answer_length=30,
):
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(
        f"Post-processing {len(examples)} example predictions split into {len(features)} features."
    )

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None  # Only used if squad_v2 is True.
        valid_answers = []

        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(
                tokenizer.cls_token_id
            )
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[
                -1 : -n_best_size - 1 : -1
            ].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or not offset_mapping[start_index]
                        or not offset_mapping[end_index]
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue
                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char:end_char],
                            "Starts" : start_char,
                            "Ends" : end_char
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[
                0
            ]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0, "Starts":-1, "Ends": -1}

        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = {"text_answer" : best_answer["text"] ,
                "starts" :  best_answer["Starts"]  ,
                "ends" : best_answer["Ends"] ,
                }
        else:
            answer = {
                "text_answer" : best_answer["text"] if best_answer["score"] > min_null_score else "",
                "starts" :  best_answer["Starts"] if best_answer["score"] > min_null_score else -1 ,
                "ends" : best_answer["Ends"] if best_answer["score"] > min_null_score else -1,
                
            }
            predictions[example["id"]] = answer
            

    return predictions

def process_name(name):
  name = unidecode(name)
  new_name = ""
  for ch in name:
    if(ch.isdigit() or ch.isalpha()):
      new_name += ch
    else:
      new_name += '_'
  return new_name

def run_model(train_dataset, test_dataset, theme):
  print(test_dataset)
  datasets = train_dataset.train_test_split(test_size=0.01)
  datasets["test"] = test_dataset

  print(datasets)

  tokenized_datasets = datasets.map(
    prepare_train_features, batched=True, remove_columns=datasets["train"].column_names
  )

  # to delete the caches of the last running model
  tf.keras.backend.clear_session()
  model = TFAutoModelForQuestionAnswering.from_pretrained(model_checkpoint)


  sen_theme = process_name(theme)
  # trainging params
  push_to_hub_model_id = f"{sen_theme}-clustered"                                  # CHANGE THIS ACCORDING TO THEME
  learning_rate = 2e-5
  num_train_epochs = 1
  weight_decay = 0.01

  # preparing the data
  train_set = model.prepare_tf_dataset(
      tokenized_datasets["train"],
      shuffle=True,
      batch_size=batch_size,
  )

  validation_set = model.prepare_tf_dataset(
      tokenized_datasets["test"],   # changed to test as I named the split column test
      shuffle=False,
      batch_size=batch_size,
  )


  
  # calculating training steps
  total_train_steps = len(train_set) * num_train_epochs

  optimizer, schedule = create_optimizer(
      init_lr=learning_rate, num_warmup_steps=0, num_train_steps=total_train_steps
  )

  model.compile(optimizer=optimizer, metrics=["accuracy"])

  push_to_hub_callback = PushToHubCallback(
      output_dir=f"./qa_model_save-{sen_theme}",
      tokenizer=tokenizer,
      hub_model_id=push_to_hub_model_id,
  )

  tensorboard_callback = TensorBoard(log_dir="./qa_model_save/logs")

  callbacks = [tensorboard_callback, push_to_hub_callback]

  # checkpoint_path = f"models/training_{sen_theme}/cp.ckpt"
  # checkpoint_dir = os.path.dirname(checkpoint_path)

  # Create a callback that saves the model's weights
  # cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,save_weights_only=True,verbose=1)


  model.fit(
      train_set,
      validation_data=validation_set,
      epochs=num_train_epochs,
      callbacks=callbacks
      # callbacks = [cp_callback]
  )

  print("done")



In [ ]:
results = {}

In [ ]:
# loading custom dataset



In [ ]:
# -------------------- Taking Theme Input and getting clustered model --------------------------#
!pip install scipy
from scipy.spatial import distance

def ret_cluster(theme):
  test_embedding = clustering_model.encode(theme)
  cluster_no = 0
  cluster_val = distance.cosine(centroids[0], test_embedding)
  for i, item in enumerate(centroids):
    # print(i)
    # print(item)
    val = distance.cosine(item, test_embedding)
    if( val < cluster_val):
      cluster_no = i
      cluster_val = val

  return cluster_no



# this is the driver code
def take_theme_input(randomly = False):
  theme_list = []
  custom_input = False
  clustering = True
  if(randomly):
    theme = squad_v1_dataset_dict.keys()
    theme_list.append(random.choice([theme_indiv for theme_indiv in squad_v1_dataset_dict.keys()]))
  elif custom_input:
    theme_str = str(input("enter the list of themes seperated by commas"))
    theme_list = theme_str.split(",")
  else:
    theme_list = list(squad_v1_dataset_dict.keys())
    print(theme_list)
  
  cnt = 0
  for theme in theme_list:
    # first take the cluster
    global model_checkpoint
    if clustering:
      cluster_no = ret_cluster(theme)
      model_checkpoint = f"nandysoham16/{cluster_no}-clustered_aug"
    else:
      model_checkpoint = "Rocketknight1/distilbert-base-uncased-finetuned-squad"

    global tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    global pad_on_right
    pad_on_right = tokenizer.padding_side=="right"

    train_dataset, test_dataset = create_dataset(theme = theme, augment_to = 300, shall_mix = True)
    run_model(train_dataset, test_dataset, theme)
    cnt+=1
    

datasets
def driver_code():
  global datasets
  datasets = load_dataset()
  split_by_themes(pd.DataFrame(datasets))
  take_theme_input()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
driver_code()

     Unnamed: 0                                           Question  \
0             0                   Which company produces the iPod?   
1             1               When was the original iPod released?   
2             2  How many different types of iPod are currently...   
3             3                   What kind of device is the iPod?   
4             4        The iPod Touch uses what kind of interface?   
..          ...                                                ...   
939         939      What type of songs was Leona Lewis known for?   
940         940  Along with mainstream AC, what format has also...   
941         941  What AC format is especially known for playing...   
942         942  When is the later of the two times that Christ...   
943         943  When is the earliest an AC station will switch...   

                        Theme  paragraph_id                Answer_text  \
0                        IPod             1                      Apple   
1          

['delete', 'wordnet', 'wordnet']
['wordnet', 'TFID', 'TFID', 'delete']
['delete', 'TFID', 'antonym', 'delete', 'antonym']
['word2vec', 'wordnet', 'word2vec', 'word2vec']
['antonym', 'antonym', 'delete']
['wordnet', 'word2vec', 'word2vec', 'word2vec']
['wordnet', 'antonym', 'word2vec', 'TFID', 'delete']
['wordnet', 'antonym', 'delete', 'wordnet', 'TFID']
['word2vec', 'antonym', 'word2vec']
['TFID', 'delete', 'antonym', 'delete']
['delete', 'antonym']
['TFID', 'word2vec', 'wordnet']
['wordnet', 'antonym', 'delete', 'word2vec']
['word2vec', 'antonym']
['TFID', 'delete', 'TFID']
['antonym', 'TFID', 'TFID', 'antonym']
['TFID', 'delete', 'TFID']
['delete', 'wordnet', 'TFID', 'word2vec', 'word2vec']
['word2vec', 'TFID']
['delete', 'delete', 'word2vec', 'antonym']
['wordnet', 'antonym', 'TFID', 'delete', 'wordnet']
['TFID', 'word2vec']
['antonym', 'delete']
['delete', 'word2vec', 'wordnet', 'wordnet']
['wordnet', 'wordnet']
['wordnet', 'antonym']
['antonym', 'wordnet', 'wordnet']
['wordnet', '

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Some layers from the model checkpoint at nandysoham16/15-clustered_aug were not used when initializing TFDistilBertForQuestionAnswering: ['dropout_379']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at nandysoham16/15-clustered_aug and are newly initialized: ['dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No loss specified in compile() - the model's internal loss computat

...

18/18 [==============================] - 806s 44s/step - loss: 0.4183 - end_logits_accuracy: 0.8611 - start_logits_accuracy: 0.8785 - val_loss: 0.2040 - val_end_logits_accuracy: 1.0000 - val_start_logits_accuracy: 1.0000
done
['antonym', 'antonym']
['word2vec', 'antonym']
['wordnet', 'TFID', 'delete']
['TFID', 'wordnet', 'TFID']
['wordnet', 'TFID']
['antonym', 'TFID', 'wordnet']
['TFID', 'wordnet']
['TFID', 'word2vec', 'delete']
['TFID', 'TFID']
['word2vec', 'delete', 'wordnet', 'antonym']
['wordnet', 'wordnet']
['antonym', 'word2vec', 'antonym']
['TFID', 'TFID', 'antonym']
['delete', 'word2vec']
['word2vec', 'antonym', 'antonym', 'TFID', 'delete']
['word2vec', 'wordnet', 'antonym', 'delete', 'word2vec']
['delete', 'antonym', 'delete', 'delete']
['wordnet', 'word2vec', 'TFID', 'antonym', 'word2vec']
['TFID', 'delete', 'wordnet', 'delete', 'delete']
['TFID', 'antonym', 'wordnet', 'word2vec']
['delete', 'TFID']
['delete', 'antonym']
['TFID', 'wordnet', 'antonym']
['TFID', 'wordnet', 'ant

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Some layers from the model checkpoint at nandysoham16/15-clustered_aug were not used when initializing TFDistilBertForQuestionAnswering: ['dropout_379']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at nandysoham16/15-clustered_aug and are newly initialized: ['dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No loss specified in compile() - the model's internal loss computat

18/18 [==============================] - 817s 45s/step - loss: 0.3270 - end_logits_accuracy: 0.8993 - start_logits_accuracy: 0.8958 - val_loss: 0.0751 - val_end_logits_accuracy: 1.0000 - val_start_logits_accuracy: 1.0000
done


In [ ]:
!zip -r models_saved.zip models

	zip warning: name not matched: models

zip error: Nothing to do! (try: zip -r models_saved.zip . -i models)
